LSTM应用股票市场初探
https://www.jianshu.com/p/8122f866436b

In [ ]:
# 导入包 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import scale 
from keras.layers import Input, Dense, LSTM, merge 
from keras.models import Model 
import pandas as pd 
import numpy as np 
import talib as ta 
import tushare as ts 
import matplotlib.pyplot as plt 
from matplotlib import rc 

rc('mathtext', default='regular') 
import seaborn as sns sns.set_style('white') 
from matplotlib import dates 
import matplotlib as mpl 
%matplotlib inline #plt.rcParams["figure.figsize"] = (20,10)


In [ ]:
# 基础参数配置 
class conf: 
    instrument = '600600' #股票代码 
    #设置用于训练和回测的开始/结束日期 
    start_date = '2005-01-01' 
    split_date = '2015-01-01' 
    end_date = '2017-05-01' 
    fields = ['close', 'open', 'high', 'low','volume','amount'] 
    # features 
    seq_len = 30 #每个input的长度 
    batch = 100 #整数，指定进行梯度下降时每个batch包含的样本数,训练时一个batch的样本会被计算一次梯度下降，使目标函数优化一步


In [ ]:
# 数据导入以及初步处理 
data = ts.get_k_data(conf.instrument, conf.start_date, conf.end_date) 
del data["code"] 
data["amount"] = data["close"]*data["volume"] 
data['return'] = data['close'].shift(-5) / data['open'].shift(-1) - 1 #计算未来5日收益率（未来第五日的收盘价/明日的开盘价） 
data.dropna(inplace=True) 
datatime = data['date'][data.date>=conf.split_date] #记录predictions的时间，回测要用 
data['return'] = data['return'].apply(lambda x:np.where(x>=0.2,0.2,np.where(x>-0.2,x,-0.2))) #去极值 
data['return'] = data['return']*10 # 适当增大return范围，利于LSTM模型训练 
data.reset_index(drop=True, inplace=True) 
scaledata = data[conf.fields] 
traindata = data[data.date<conf.split_date]
